# Write MulVal Rules from Ontology 
### Purpose: Test read from / write to datalog
### Author: @mjz


## Background/Sec0


In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from itertools import chain
from jinja2 import Template
import argparse

import os
import platform
import sys
import logging
logging.basicConfig(filename='cat-dog.log',level=logging.DEBUG)

import jupyter_core

# from owlready2 import *
from stix2 import *
from stix2 import FileSystemSource as fs
from stix2 import Filter
from stix2.utils import get_type_from_id

from pyxsb import pyxsb_start_session, pyxsb_end_session, pyxsb_command, \
                  pyxsb_query, XSBFunctor, XSBVariable, xsb_to_json, json_to_xsb

from pyxsb import *


sys.path.append('..')

# STIX Data Loading
cti_home = '../mitre/cti/'
enterprise = cti_home + 'enterprise-attack'
mobile = cti_home + 'mobile-attack'
pre = cti_home + 'pre-attack'
capec = cti_home + 'capec'

e = fs(enterprise)
c = fs(capec)
p = fs(pre)
m = fs(mobile)

sources = [e, c, p, m]
all_ds = CompositeDataSource()
all_ds.add_data_sources([e, m, p, c])

# MulVal Data Loading
BASE_DIR = '/opt/projects/diss/jupyter_nbs/mine'
DATA_DIR = 'data'
WORKING_DIR = '/'.join((BASE_DIR, DATA_DIR, 'test_003'))

XSB_ARCH_DIR = '/opt/apps/xsb/XSB/config/x86_64-unknown-linux-gnu'



os.chdir(WORKING_DIR)

%pwd


'/opt/projects/diss/jupyter_nbs/mine/data/test_003'

# Port MulVal to Python (graph_gen.sh)
```
 mimic the bash call, accepting the same args (as time permits) and producing the same output
    
    previous args:
    ------------------------
    graph_gen.sh 
        [-r|--rule rulefile]                                                                                                    
        [-a|--additional additional_rulefile]                                                                                   
        [-c|--constraint constraint_file]                                                                                                   
        [-g|--goal goal]                                                                                                                    
        [-d|--dynamic dynamic_file]                                                                                                         
        [-p]                                                                                                                                
        [-s|--sat]                                                                                                                          
        [-t|--t trace_option]                                                                                                               
        [-tr|--trim]                                                                                                                        
        [-v|--visualize 
        [--arclabel] 
        [--reverse]]                                                                                           
        [--cvss]                                                                                                                
        [-h|--help]                                                                                                                   
        [attack_graph_options]                                                                                                        
        input_file        
```

In [10]:
## port MulVals graph_gen.sh to python
MULVALROOT = '/opt/mulval'
INTERACTIONRULES = '/'.join((MULVALROOT, 'kb/interaction_rules.P'))
INTERACTIONRULES_CVSS =  '/'.join((MULVALROOT,  'kb/interaction_rules_with_metrics.P'))
RULES_WITH_METRIC_ARTIFACTS = '/'.join((MULVALROOT, 'kb/interaction_rules_with_metric_artifacts.P'))

_RULE_FILES = list()
_RULE_FILES_ADDITIONAL = list()
RUNNING_RULES_NAME = 'running_rules.P'

trace_option = 'completeTrace2'

INPUT_FILE = 'input.P'

if __name__ == "__main__":
    """
    all the flags
    """
    
    arg_parser = argparse.ArgumentParser(description='Process MulVal flags')
    arg_parser.add_argument('--rulefile',  '-r', action='append',  # allow multiple rule files
                        help='add rulefile(s) -r rulefile.txt')
    arg_parser.add_argument('--additional',  '-a', action='append',  # allow multiple rule files
                        help='add additional rulefile(s) -a anotherrulefile.txt')
    arg_parser.add_argument('--constraint',  '-c', action='append',  # allow multiple rule files
                        help='add constraint files(s) -c constraintfile.txt')    
    arg_parser.add_argument('--goal',  '-g', action='append',  # allow multiple goals
                        help='add goals -g goal')
    arg_parser.add_argument('--dynamic',  '-d', action='append',  # allow multiple dynamic files
                        help='add dynamic files -d dynamicfile.txt')
    arg_parser.add_argument('--visualize',  '-v', help='create viz (implies csv output)', action='store_true')
    arg_parser.add_argument(  '-l', help='CSV OUTPUT', action='store_true')
#     arg_parser.add_argument('viz_options', choices=['--arclabel', '--reverse', '--simple', '--nometric']), action='append'
    arg_parser.add_argument('--arclabel', help='viz_options', action='store_true')
    arg_parser.add_argument('--reverse', help='viz_options', action='store_true')
    arg_parser.add_argument('--simple', help='viz_options', action='store_true')
    arg_parser.add_argument('--nometric', help='viz_options', action='store_true')
    
    
    arg_parser.add_argument('--sat',  '-s', help='SAT', action='store_true')
    arg_parser.add_argument('--satgui',  '-sg', help='SAT GUI', action='store_true')
    arg_parser.add_argument('--trace',  '-t', help='trace option')
    arg_parser.add_argument('--trim',  '-tr', help='trim option', action='store_true')
    arg_parser.add_argument('--trimdom',  '-td', help='trimdom option', action='store_true')
    arg_parser.add_argument('--cvss', help='cvss option', action='store_true')
    arg_parser.add_argument(  '-ma', help='metric artifacts', action='store_true')
    ### @TODO figure out what all these do...
    
    args, other_args = arg_parser.parse_known_args()
    print('args: ', args)
    print('other args: ', other_args)
    
#     graph_gen()  # ...
    
    
    
def graph_gen(*args, **kwargs):
    """
    do the things graph_gen.sh does
    """
    _input_file = INPUT_FILE if 'input_file' not in kwargs else kwargs.get('input_file')
    _MULVALROOT = MULVALROOT  # if 'MULVALROOT' not in kwargs else kwargs.get('MULVALROOT')
    _type = None  #if 'type' not in kwargs else kwargs.get('type')  # 'run' | 'environment' includes the mulval_run line
    _tracemode = 'completeTrace2' if 'tracemode' not in kwargs else kwargs.get('tracemode')
    _dynamic_file = None
    _trim = False  # True is --trim | -tr flags passed
    _trim_rules = MULVALROOT + '/src/analyzer/advances_trim.P'
    _no_trim_rules = MULVALROOT + '/src/analyzer/advances_notrim.P'
    _cvss = False  # original script tests if this is zero (-z $CVSS) so this is probably a path not bool
    _goal = None  # goal passed in a flag
    # template this out for later
    # vars:
    
    ts = """
:-['{{ _MULVALROOT }}/lib/libmulval'].  % start base run script
:-['{{ _MULVALROOT }}/src/analyzer/translate'].
:-['{{ _MULVALROOT }}/src/analyzer/attack_trace'].
:-['{{ _MULVALROOT }}/src/analyzer/auxiliary'].

:-dynamic meta/1.

:-load_dyn('running_rules.P').

:-load_dyn('{{_input_file}}').

:-assert(traceMode({{_tracemode}})).  % end base run script

% set if dynamic changes file is set (-d flag)
{{':-load_dyn({{_dynamic_file}}).
:-apply_dynamic_changes.' if _dynamic_file }}

% set if --trim | -tr flag passed
{% if _trim %}
:-load_dyn('{{_trim_rules}}').                                                                                                                            
:-tell('edges').
:-writeEdges.
:-told.
:-shell('rm -f dominators.P').
:-shell('dom.py edges dominators.P').
:-loadDominators('dominators.P'). 
{% else %}
% else set if no --trim | -tr flag passed
:-load_dyn('{{_no_trim_rules }}'). 
{% endif %}

% add this line if CVSS flag is not set (non-zero len) 
% @TODO should expect a string here not bool
{{':-assert(cvss(_, none)).' if not _cvss}}

% add goal if passed as a flag
{{':- assert(attackGoal(_goal)).' if _goal }}

% add mulval run if we're not writing the environment program
{{':-mulval_run.' if _type == 'run' }}

"""

    logging.info('writing rule file...')
    _RULE_FILES.append(INTERACTIONRULES) # @TODO cvss and ma checks
    writeRulesFile(_RULE_FILES, _RULE_FILES_ADDITIONAL)
    
    logging.info('writing environment.P...')
    tm = Template(ts)
    logging.debug(locals())
    writeFile('environment.P', tm.render(locals()))
    
    logging.info('writing run.P...')
    writeFile('run.P', tm.render(locals(), _type='run'))
    
    logging.info('running mulval in xsb...')
    runMulVal()

def writeRulesFile(_RULE_FILES, _RULE_FILES_ADDITIONAL):
    """@TODO needs logic for placement, tabling, validation"""
    
    with open(RUNNING_RULES_NAME, 'w+') as outfile:
        for fname in chain(_RULE_FILES, _RULE_FILES_ADDITIONAL):
            with open(fname, 'r') as infile:
                outfile.write(infile.read())
    
    
def writeFile(file_name, file_text, mode='w+'):
    '''
    w  write mode
    r  read mode
    a  append mode
    w+  create file if it doesn't exist and open it in (over)write mode
        [it overwrites the file if it already exists]
    r+  open an existing file in read+write mode
    a+  create file if it doesn't exist and open it in append mode
    '''
    with open(file_name, mode) as file:
        file.write(file_text)
        

    
def runMulVal():
    pyxsb_start_session(XSB_ARCH_DIR)
#     from pyxsb import *

    pyxsb_query('catch(abort,Exception,true).')

    
    # xsb 2>xsb_log.txt 1>&2 <<EOF                                                                                                                
    # [environment].                                                                                                                              
    # tell('goals.txt').                                                                                                                          
    # writeln('Goal:').                                                                                                                           
    # iterate(attackGoal(G),                                                                                                                      
    #         (write(' '), write_canonical(G), nl)).                                                                                              
    # told.  
    # # tabling breaks the  pyxsb_command but works with lowlevel api :?
    # UPDATE: 3.7 fails... rolling back to 3.6 works
    c2p_functor(b"consult", 1, reg_term(1))
    c2p_string(b"environment",p2p_arg(reg_term(1),1))
    xsb_command()

    c2p_functor(b"tell", 1, reg_term(1))
    c2p_string(b"goals.txt",p2p_arg(reg_term(1),1))
    xsb_command()

    pyxsb_command('writeln("Goal:"). ')

    c2p_functor(b"iterate", 1, reg_term(1))
    c2p_string(b"attackGoal(G),(write(' '), write_canonical(G), nl)",p2p_arg(reg_term(1),1))
    xsb_command()
    pyxsb_command('told.')
#     pyxsb_end_session()
    
#     pyxsb_start_session(XSB_ARCH_DIR)
    c2p_functor(b"consult", 1, reg_term(1))
    c2p_string(b"run",p2p_arg(reg_term(1),1))
    xsb_command()
#     pyxsb_command('[run].')

    pyxsb_end_session()

args:  Namespace(additional=None, arclabel=False, constraint=None, cvss=False, dynamic=None, goal=None, l=False, ma=False, nometric=False, reverse=False, rulefile=None, sat=False, satgui=False, simple=False, trace=None, trim=False, trimdom=False, visualize=False)
other args:  ['-f', '/home/toor/.local/share/jupyter/runtime/kernel-d83f93b8-ef03-4b02-9748-5072218ddd8b.json']


In [11]:


graph_gen()



        
# def makeRunFile(*args, **kwargs):
    
#     tm = Template(
# """:-['/opt/mulval/lib/libmulval'].
# :-['/opt/mulval/src/analyzer/translate'].
# :-['/opt/mulval/src/analyzer/attack_trace'].
# :-['/opt/mulval/src/analyzer/auxiliary'].

# :-dynamic meta/1.
# :-load_dyn('running_rules.P').
# :-load_dyn('{{input_file}}').
# :-assert(traceMode(completeTrace2)).
# :-load_dyn('/opt/mulval/src/analyzer/advances_notrim.P').
# :-apply_dynamic_changes. 
# :-assert(cvss(_, none)).

# :-mulval_run.
# """)
#     writeFile('run.P', tm.render(input_file=input_file))
#     writeFile('environment.P', tm.render(input_file=input_file))

In [14]:

mulval_run_text = """
:-['/opt/mulval/lib/libmulval'].
:-['/opt/mulval/src/analyzer/translate'].
:-['/opt/mulval/src/analyzer/attack_trace'].
:-['/opt/mulval/src/analyzer/auxiliary'].

:-dynamic meta/1.

:-load_dyn('running_rules.P').

:-load_dyn('input.P').

:-assert(traceMode(completeTrace2)).

:-load_dyn('/opt/mulval/src/analyzer/advances_notrim.P').

:-assert(cvss(_, none)).

:-mulval_run.

"""

file_name = 'run.P'
f = open(file_name, 'w+')  # open file in overwrite mode
f.write(mulval_run_text)
f.close()

input_p_text = """
attackerLocated(internet).
attackGoal(execCode(workStation,_)).

hacl(internet, webServer, tcp, 80).
hacl(webServer, _,  _, _).
hacl(fileServer, _, _, _).
hacl(workStation, _, _, _).
hacl(H,H,_,_).

/* configuration information of fileServer */
networkServiceInfo(fileServer, mountd, rpc, 100005, root).
nfsExportInfo(fileServer, '/export', _anyAccess, workStation).
nfsExportInfo(fileServer, '/export', _anyAccess, webServer).
vulExists(fileServer, vulID, mountd).
vulProperty(vulID, remoteExploit, privEscalation).
localFileProtection(fileServer, root, _, _).

/* configuration information of webServer */
vulExists(webServer, 'CAN-2002-0392', httpd).
vulProperty('CAN-2002-0392', remoteExploit, privEscalation).
networkServiceInfo(webServer , httpd, tcp , 80 , apache).

/* configuration information of workStation */
nfsMounted(workStation, '/usr/local/share', fileServer, '/export', read).
"""

file_name = 'input.P'
f = open(file_name, 'w+')  # open file in overwrite mode
f.write(input_p_text)
f.close()

ft_txt = """
% prolog

child(anne,bridget).
child(bridget,caroline).
child(caroline,donna).
child(donna,emily).

descend(X,Y) :- child(X,Y).
descend(X,Y) :- child(X,Z), descend(Z,Y).

label('günter', "Günter").
label(anne, "Anne").
label(bridget, "Bridget").
label(caroline, "Caroline").
label(donna, "Donna").
"""
file_name = 'ft.P'
f = open(file_name, 'w+')  # open file in append mode
f.write(ft_txt)
f.close()


## Run MulVal

In [14]:
pyxsb_start_session(XSB_ARCH_DIR)


pyxsb_end_session()

In [112]:


pyxsb_start_session(XSB_ARCH_DIR)
from pyxsb import *

pyxsb_query('catch(abort,Exception,true).')


c2p_functor(b"consult", 1, reg_term(1))
c2p_string(b"environment",p2p_arg(reg_term(1),1))
xsb_command()

c2p_functor(b"tell", 1, reg_term(1))
c2p_string(b"goals.txt",p2p_arg(reg_term(1),1))
xsb_command()

pyxsb_command('writeln("Goal:"). ')

c2p_functor(b"iterate", 1, reg_term(1))
c2p_string(b"attackGoal(G),(write(' '), write_canonical(G), nl)",p2p_arg(reg_term(1),1))
xsb_command()
pyxsb_command('told.')

# pyxsb_command('iterate(attackGoal(G),(write(' '), write_canonical(G), nl)).')
# pyxsb_command('told.')


# pyxsb_command('[environment].')
# pyxsb_command('tell("goals.txt").')

c2p_functor(b"consult", 1, reg_term(1))
c2p_string(b"run",p2p_arg(reg_term(1),1))
xsb_command()

# if xsb_command_string("consult(run)."):
#     raise Exception ("Error (string) consulting basics.")

# pyxsb_command('consult("running_rules.P").')
# pyxsb_query('consult("running_rules.P").')
# pyxsb_command('[run].')
# pyxsb_query(u'[run].')

# for row in pyxsb_query('hacl(A, B, C, D).'):
#     print(u"hacl %s %s %s %s" % (row[0], row[1], row[2], row[3]))

# for row in pyxsb_query('hacl(A, B, C, D).'):
#     print(u"hacl %s %s" % (row[0], row[1]))
    
# for row in pyxsb_query(u'findall(Goal, attack_simulation(Goal), L).'):
#     print(u"hacl %s %s" % (row[0], row[1]))
    
# print(pyxsb_query(u'findall(Goal, attack_simulation(Goal), L).'))
# pyxsb_query('catch(abort,Exception,true).')

pyxsb_end_session()
%pwd

'/opt/projects/diss/jupyter_nbs/mine/data/test_003'

In [16]:

running_rules = """

/******************************************************/
/****         Predicates Declaration              *****/
/******************************************************/

primitive(inCompetent(_principal)).
primitive(competent(_principal)).
primitive(clientProgram(_host, _programname)).
primitive(vulExists(_host, _vulID, _program)).
primitive(vulProperty(_vulID, _range, _consequence)).
primitive(hacl(_src, _dst, _prot, _port)).
primitive(attackerLocated(_host)).
primitive(hasAccount(_principal, _host, _account)).
primitive(networkServiceInfo(_host, _program, _protocol, _port, _user)).
primitive(setuidProgramInfo(_host, _program, _owner)).
primitive(nfsExportInfo(_server, _path, _access, _client)).
primitive(nfsMounted(_client, _clientpath, _server, _serverpath, _access)).
primitive(localFileProtection(_host, _user, _access, _path)).
primitive(dependsOn(_h, _program, _library)).
primitive(installed(_h, _program)).
primitive(bugHyp(_,_,_,_)).
primitive(vulExists(_machine,_vulID,_program,_range,_consequence)).
primitive(canAccessFile(_host, _user, _access, _path)).
primitive(isWebServer(_host)).
meta(cvss(_vulID, _ac)).


derived(execCode(_host, _user)).
derived(netAccess(_machine,_protocol,_port)).
derived(canAccessHost(_host)).
derived(accessFile(_machine,_access,_filepath)).
derived(accessMaliciousInput(_host, _principal, _program)).
derived(principalCompromised(_victim)).
derived(dos(_host)).
derived(logInService(_host, _protocol, _port)).

meta(attackGoal(_)).
meta(advances(_, _)).


"""
file_name = "running_rules.P"
f = open(file_name, 'w+')  # open file in overwrite mode
f.write(running_rules)
f.close()



## Sec2

and some more text

In [ ]:


/******************************************************/
/****         Tabling Predicates                  *****/
/*   All derived predicates should be tabled          */
/******************************************************/

:- table execCode/2.
:- table netAccess/3.
:- table canAccessHost/1.
:- table canAccessFile/4.
:- table accessFile/3.
:- table principalCompromised/1.
:- table vulExists/5.
:- table logInService/3.



/******************************************************/
/****         Interaction Rules                   *****/
/******************************************************/

/****** Section execCode ******
interaction_rule(
   (execCode(H, Perm) :-
	hasAccount(P, H, Perm)),
   rule_desc('Insider threat', 1)).
*/

interaction_rule(
   (execCode(Host, Perm) :-
	principalCompromised(Victim),
	hasAccount(Victim, Host, Perm),
	canAccessHost(Host)),
   rule_desc('When a principal is compromised any machine he has an account on will also be compromised',
   0.5)).

interaction_rule(
  (execCode(Host, root) :-
	execCode(Host, _Perm2),
	vulExists(Host, _, Software, localExploit, privEscalation)),
  rule_desc('local exploit',
  1.0)).

interaction_rule(
  (execCode(H, Perm) :-
	vulExists(H, _, Software, remoteExploit, privEscalation),
	networkServiceInfo(H, Software, Protocol, Port, Perm),
	netAccess(H, Protocol, Port)),
  rule_desc('remote exploit of a server program',
  1.0)).

interaction_rule(
  (execCode(H, Perm) :-
        vulExists(H, _, Software, remoteClient, privEscalation),
	hasAccount(Victim, H, Perm),
        accessMaliciousInput(H, Victim, Software)),
  rule_desc('remote exploit for a client program',
  0.5)).

interaction_rule(
  (execCode(H, root) :-
	accessFile(H, write, _Path)),
  rule_desc('Trojan horse installation',
  0.8)).

/* Singleton variable at head
interaction_rule(
 (execCode( Attacker, Host, _) :-
   execCode(Attacker, Host, root)),
  'execution at any level if root execution').
*/



/******** Section netAccess ********/
/* accessing a host through network according to a hacl policy.
   For now we assume that every user on a local
   machine has access to network. this may change
   later. */
interaction_rule(
  (netAccess(H2, Protocol, Port) :-
	execCode(H1, _Perm),  /* Any permission level */
	advances(H1, H2),
    hacl(H1, H2, Protocol, Port)),
  rule_desc('multi-hop access',
  0.5)).

interaction_rule(
  (netAccess(H, Protocol, Port) :-
	attackerLocated(Zone),
	hacl(Zone, H, Protocol, Port)),
  rule_desc('direct network access',
  1.0)).

interaction_rule(
  (netAccess(H, Protocol, Port) :-
	attackerLocated(H)),
  rule_desc('direct on-host access',
  1.0)).


/****** Section canAccessHost ******/
interaction_rule(
  (canAccessHost(H) :-
	execCode(H, _Perm)),
  rule_desc('Access a host through executing code on the machine',
  1.0)).

interaction_rule(
  (canAccessHost(H) :-
	logInService(H, Protocol, Port),
	netAccess(H, Protocol, Port)),
  rule_desc('Access a host through a log-in service',
  1.0)).


/******** Section accessFile ********/
interaction_rule(
  (accessFile(H, Access, Path) :-
	execCode(H, Usr),
	canAccessFile(H, Usr, Access, Path)),
  rule_desc('execCode implies file access',
  1.0)).


/****** Section principalCompromised ******/
interaction_rule(
  (principalCompromised(Victim) :-
	hasAccount(Victim, H, _Perm),
	execCode(H, root)),
  rule_desc('password sniffing',
  0.8)).

interaction_rule(
  (principalCompromised(Victim) :-
	hasAccount(Victim, H, User),
	execCode(H, User)),
  rule_desc('password sniffing',
  0.8)).

/*
interaction_rule(
  (principalCompromised(Victim) :-
	inCompetent(Victim)),
  rule_desc('incompetent user',
  0.2)).
*/



/********************************************************/
/*      Software specific knowledge                     */
/********************************************************/

/*
explain(logInService(H, Protocol, Port), Text) :-
	fmt_write_string(Text,
  "There is a login service running under protocol %S and port %S on host %S.", args(Protocol, Port, H)).
*/



/***************** Section ssh **********************/
interaction_rule(
  (logInService(H, Protocol, Port) :-
	networkServiceInfo(H, sshd, Protocol, Port, _)),
  rule_desc('',
  1)).

interaction_rule(
  (logInService(H, Protocol, Port) :-
	networkServiceInfo(H, vpnService, Protocol, Port, _)),
  rule_desc('',
  1)).


/**************** Section  nfs *****************/
/* Principal P can access files on a NFS server if the files
   on the server are mounted at a client and he can access the
   files on the client side */
interaction_rule(
  (accessFile(Server, Access, ServerPath) :-
	nfsMounted(Client, ClientPath, Server, ServerPath, Access),
	accessFile(Client, Access, ClientPath)),
  rule_desc('NFS semantics',
  1)).


/* Principal P can access files on a NFS client if the files
   on the server are mounted at the client and he can access the
   files on the server side */
interaction_rule(
  (accessFile(Client, Access, ClientPath) :-
	nfsMounted(Client, ClientPath, Server, ServerPath, read),
	accessFile(Server, Access, ServerPath)),
  rule_desc('NFS semantics',
  1)).


interaction_rule(
  (accessFile(Server, Access, Path) :-
	execCode(Client, _User),
    nfsExportInfo(Server, Path, Access, Client),
    hacl(Client, Server, nfsProtocol, nfsPort)),
  rule_desc('NFS shell',
  0.8)).


interaction_rule(
  (canAccessFile(H, Usr, Acc, Path) :-
	localFileProtection(H, Usr, Acc, Path)),
  rule_desc('',
  1)).


/* Singleton variable in head
interaction_rule(
  (canAccessFile(_H, root, _Access, _Path)),
  'root has arbitrary access').
*/


interaction_rule((vulExists(H, ID, Sw, Range, Consequence):-
	        vulExists(H, ID, Sw),
		vulProperty(ID, Range, Consequence)),
             rule_desc('',
             1)).

interaction_rule((vulExists(H, _ID, Sw, Range, Consequence):-
	        bugHyp(H, Sw, Range, Consequence)),
             rule_desc('Introducing hypothetical bug',
             1)).


interaction_rule((vulExists(H, ID, Sw, Range, Consequence):-
	        vulExists(H, ID, Library, Range, Consequence),
		dependsOn(H, Sw, Library)),
             rule_desc('Library bug',
             1)).


interaction_rule(
   (accessMaliciousInput(H, Victim, Software) :-
     inCompetent(Victim),
     hacl(H, MaliciousMachine, httpProtocol, httpPort),
     attackerLocated(MaliciousMachine)),
  rule_desc('Browsing a malicious website', 0.8)).

interaction_rule(
   (accessMaliciousInput(H, Victim, Software) :-
     competent(Victim),
     hacl(H, MaliciousMachine, httpProtocol, httpPort),
     attackerLocated(MaliciousMachine)),
  rule_desc('Browsing a malicious website', 0.1)).

interaction_rule(
   (accessMaliciousInput(H, Victim, Software) :-
     inCompetent(Victim),
     isWebServer(CompromisedMachine),
     hacl(H, CompromisedMachine, httpProtocol, httpPort),
     execCode(CompromisedMachine, _)),
  rule_desc('Browsing a compromised website', 0.4)).


/*
interaction_rule(
   (canAccessMaliciousInput(H, Browser) :-
     installed(H, Browser),
     isWebBrowser(Browser)),
  rule_desc('A browser can potentially access malicious input',
  1)).


interaction_rule(
   (canAccessMaliciousInput(H, Software) :-
	vulExists(H, _, Software, remoteClient, privEscalation),
	inCompetent(Victim),
	hasAccount(Victim, H, _Perm)),
  rule_desc('A remote client vulnerability can potentially access malicious input from a host used by careless user',
  1)).


interaction_rule(
   (canAccessMaliciousInput(H, Browser) :-
     installed(H, Browser),
     isWebBrowser(Browser),
     hacl(H, MaliciousMachine, httpProtocol, httpPort),
     attackerLocated(MaliciousMachine)),
  rule_desc('Browsing a malicious website',
  1)).

interaction_rule(
   (canAccessMaliciousInput(H, Browser) :-
     installed(H, Browser),
     isWebBrowser(Browser),
     hacl(H, CompromisedMachine, httpProtocol, httpPort),
     execCode(CompromisedMachine, _)),
  rule_desc('Browsing a compromised website',
  0.4)).

interaction_rule(
   (canAccessMaliciousInput(H, EmailClientSoftware) :-
     installed(H, EmailClientSoftware),
     isEmailClient(EmailClientSoftware),
     isEmailServer(EmailServerSoftware),
     hacl(H, EmailServer, EmailProtocol, EmailPort),
     networkServiceInfo(EmailServer, EmailServerSoftware,
                                EmailProtocol, EmailPort, _Perm)),
   rule_desc('receive an email message',
   0.4)).

*/

primitive(networkServiceInfo(_host, _program, _protocol, _port, _perm)).
primitive(gateway(_host)).
primitive(flowExists(_src, _dst, _protocol, _port, _user)).


    interaction_rule(
  (netAccess(H2, _protocol, _port) :-
                gateway(H1), 
               advances(H1, H2),
               netAccess(H1, _protocol, _port),
    hacl(H1, H2, _protocol, _port)),
  rule_desc('multi-hop access by gateway', 0.5)).


  
  /*The following rule was added to enable arp spoofing*/
  interaction_rule(
  (principalCompromised(Victim) :-
                hasAccount(Victim, RemoteHost, User),
                /* Arp spoof works only if the victim and attacker are in the same subnet*/
                attackerLocated(Zone),
                hacl(Zone, H, _anyProtocol, _anyPort),
                /* Victim is using standard arp for address resolution*/
                networkServiceInfo(H, arpd, _protocol, _port, _),
                /* The standard arpd protocol is vulnerable to spoofing */
                vulExists(H, arpSpoofVuln, arpd, remoteExploit, arpSpoof),
                /* The User has an account on a login service on the remote host */
                logInService(RemoteHost, Protocol, Port),
                /* There is an active connection from the host to the remote machine */
                flowExists(H, RemoteHost, Protocol, Port, User)), 
  rule_desc('password sniffing through spoof',
  0.8)).
  
interaction_rule(
  (logInService(H, Protocol, Port) :-
                 networkServiceInfo(H, ftpd, Protocol, Port, _)),
                rule_desc('log in for ftpd',
                1)).

## Sec3

and some more text

## Sec4

and some more text

## Refs

and some more text